<a href="https://colab.research.google.com/github/albeeertomoreno/03-MIAR-Algoritmos-de-Optimizacion/blob/main/Algoritmos_TrabajoPr%C3%A1ctico_Alberto_Moreno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Alberto Moreno Torreira  <br>
Url:<br>
Google Colab:https://colab.research.google.com/drive/1-mjY_xoUJUvHpRdeKIQNfQygKn92lckN?usp=sharing <br>
Problema:
>1. Sesiones de doblaje

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

....







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

Antes de poder desarrollar el código es necesario instalar la librería que vamos a utilizar.

In [ ]:
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

Antes de buscar el código que minimice los costes de los actores por desplazarse al rodaje, cargamos los datos desde el Drive.

In [ ]:
# Permitir acceso al drive
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


# Especifica la ruta en tu unidad de Google Drive al archivo Excel
ruta = '/content/drive/My Drive/Colab_Notebooks/AG_OPT/res/datos_practica1.xlsx'

nombres_columnas = ['Toma', 'actor1', 'actor2', 'actor3', 'actor4', 'actor5',
                    'actor6', 'actor7', 'actor8', 'actor9', 'actor10', 'NaN',
                    'Total']


# Lee el archivo Excel en un DataFrame de Pandas
datos_lista = pd.read_excel(ruta,names=nombres_columnas )
# Elimina la primera fila del DataFrame
datos_lista.drop([0,31,32], axis=0,inplace=True)
del datos_lista['NaN']

display(datos_lista)

Mounted at /content/drive


,Toma,actor1,actor2,actor3,actor4,actor5,actor6,actor7,actor8,actor9,actor10,Total
1,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,5
2,2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3
3,3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,3
4,4,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,4
5,5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,3
6,6,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,4
7,7,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,4
8,8,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3
9,9,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3
10,10,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,4


In [ ]:
##Respuesta
from ortools.linear_solver import pywraplp

# Datos de entrada
n_tomas = 30  # Número total de tomas
n_actores = 10  # Número total de actores
max_tomas_por_dia = 6  # Máximo número de tomas que se pueden grabar por día
n_dias = n_tomas  # Máximo teórico de días, podría ser menor

# Convertir el DataFrame datos a una lista de listas para las restricciones de actores
datos = datos_lista.iloc[:, 1:-1].values.tolist()

# Crear el modelo de optimización
solver = pywraplp.Solver.CreateSolver('SCIP')

# Variables
x = {}  # x[toma][dia] = 1 si la toma se realiza en ese día
y = {}  # y[actor][dia] = 1 si el actor trabaja ese día
for toma in range(n_tomas):
    for dia in range(n_dias):
        x[toma, dia] = solver.IntVar(0, 1, f'x[{toma},{dia}]')
for actor in range(n_actores):
    for dia in range(n_dias):
        y[actor, dia] = solver.IntVar(0, 1, f'y[{actor},{dia}]')

# Restricciones
# No más de max_tomas_por_dia tomas por día
for dia in range(n_dias):
    solver.Add(solver.Sum(x[toma, dia] for toma in range(n_tomas)) <= max_tomas_por_dia)

# Cada toma se asigna al menos una vez
for toma in range(n_tomas):
    solver.Add(solver.Sum(x[toma, dia] for dia in range(n_dias)) == 1)

# Si una toma se realiza en un día, todos los actores necesarios deben estar presentes
for toma in range(n_tomas):
    for dia in range(n_dias):
        actores_necesarios = [actor for actor in range(n_actores) if datos[toma][actor] == 1]
        for actor in actores_necesarios:
            solver.Add(x[toma, dia] <= y[actor, dia])
# Función objetivo: minimizar el número total de días de trabajo de todos los actores
solver.Minimize(solver.Sum(y[actor, dia] for actor in range(n_actores) for dia in range(n_dias)))

# Resolver el modelo
status = solver.Solve()

# Verificar si se encontró una solución óptima
if status == pywraplp.Solver.OPTIMAL:
    print('Solución:')
    for dia in range(n_dias):
        print(f'Día {dia + 1}:', end=' ')
        for toma in range(n_tomas):
            if x[toma, dia].solution_value() > 0.5:
                print(f'Toma {toma + 1}', end=' ')
        print()
    print('Número total de días de trabajo de los actores:', solver.Objective().Value())
else:
    print('No se encontró una solución óptima.')

No se encontró una solución óptima.


El método planteado de programación lineal entera resulta muy costoso por lo que buscaremos un método heurístico que resuelva el problema. Aunque la solución no sea la óptimo podemos acercarnos a ella.

In [ ]:
import pandas as pd
from itertools import combinations, product


# Número máximo de tomas por día
max_tomas_por_dia = 6

# Inicialización de la planificación
planificacion = []
tomas_disponibles = set(datos_lista['Toma'])

# Calcular la participación de cada actor
participacion_actor = {actor: set() for actor in range(1, 11)}
for index, row in datos_lista.iterrows():
    for actor in range(1, 11):
        if row[f'actor{actor}'] > 0:
            participacion_actor[actor].add(row['Toma'])

# Ordenar actores por número de participaciones
actores_ordenados = sorted(participacion_actor, key=lambda x: len(participacion_actor[x]), reverse=True)

# Planificar tomas
while tomas_disponibles:
    mejor_combinacion = []
    mejor_cubrimiento = -1

    # Combinaciones de tomas para un día
    for combinacion in combinations(tomas_disponibles, min(max_tomas_por_dia, len(tomas_disponibles))):
        cubrimiento = sum([len(participacion_actor[actor].intersection(combinacion)) for actor in actores_ordenados])
        if cubrimiento > mejor_cubrimiento:
            mejor_cubrimiento = cubrimiento
            mejor_combinacion = combinacion

    # Actualizar planificación y tomas disponibles
    if mejor_combinacion:
        planificacion.append(mejor_combinacion)
        tomas_disponibles.difference_update(mejor_combinacion)
        for actor in actores_ordenados:
            participacion_actor[actor].difference_update(mejor_combinacion)

# Formatear la salida
dias_de_grabacion = len(planificacion)
resultado_formato = f"Número total de días de grabación: {dias_de_grabacion}\n"

for i, dia in enumerate(planificacion, start=1):
    tomas_del_dia = ', '.join(map(str, dia))
    resultado_formato += f"Día {i}: Tomas {tomas_del_dia}\n"

# Mostrar el resultado
print(resultado_formato)



Número total de días de grabación: 5
Día 1: Tomas 1, 4, 6, 7, 11, 12
Día 2: Tomas 2, 10, 20, 22, 25, 26
Día 3: Tomas 3, 5, 8, 9, 13, 14
Día 4: Tomas 15, 16, 17, 18, 19, 29
Día 5: Tomas 21, 23, 24, 27, 28, 30



Respondiendo a las preguntas planteadas podemos distinguir en el código:
- Espacio de Soluciones:

El espacio de soluciones es el conjunto de todas las combinaciones posibles de tomas asignadas a cada día de grabación.
Las líneas que contribuyen a explorar este espacio de soluciones son las que generan las combinaciones de tomas para un día:

    

    for combinacion in combinations(tomas_disponibles, min(max_tomas_por_dia, len(tomas_disponibles))):

  

- Función Objetivo:

La función objetivo es maximizar el cubrimiento de participación de actores en las tomas seleccionadas para cada día (minimizar los dias que un actor tiene que ir a rodaje).
La parte del código que calcula la función objetivo es:

    

    cubrimiento = sum([len(participacion_actor[actor].intersection(combinacion)) for actor in actores_ordenados])
    if cubrimiento > mejor_cubrimiento:
        mejor_cubrimiento = cubrimiento
        mejor_combinacion = combinacion

    

- Implementación de Restricciones:

Las restricciones se implementan en varias partes del código:

-Restricción de número máximo de tomas por día:

  

    min(max_tomas_por_dia, len(tomas_disponibles))

Esta línea asegura que no se exceda el número máximo de tomas por día.

-Restricción de disponibilidad de actores:


    participacion_actor = {actor: set() for actor in range(1, 11)}
    for index, row in datos_lista.iterrows():
        for actor in range(1, 11):
            if row[f'actor{actor}'] > 0:
                participacion_actor[actor].add(row['Toma'])

Se crea un diccionario que mapea cada actor a las tomas en las que participa, asegurándose de que las tomas asignadas a un día incluyan solo a los actores disponibles para esas tomas.

-Actualización de tomas disponibles y participación de actores:


    tomas_disponibles.difference_update(mejor_combinacion)
    for actor in actores_ordenados:
       participacion_actor[actor].difference_update(mejor_combinacion)

Estas líneas actualizan las tomas disponibles y la participación de los actores después de cada día de grabación, lo cual es crucial para respetar las restricciones en los días siguientes.



#Análisis
¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

- Complejidad de Tiempo:
El algoritmo utiliza la función combinations de Python, que genera todas las combinaciones posibles de un tamaño dado (max_tomas_por_dia) a partir de un conjunto de elementos (tomas_disponibles). La cantidad de combinaciones generadas es $\binom{n}{k}$
, donde n es el número total de tomas y k es el máximo número de tomas por día. La fórmula del coeficiente binomial es $\binom{n}{k} = \frac{n!}{k!(n-k)!}$
 lo que implica una complejidad temporal de O(n!) en el peor caso.




- Espacio de Soluciones

 En este caso, es el número total de combinaciones posibles de tomas, que, como se mencionó anteriormente, es $\binom{n}{k}$.  Será $\binom{30}{6}$ para el primer reparto (primer día). $\binom{24}{6}$ (segundo día)  $\binom{18}{6}$ (tercer día) $\binom{12}{6}$ (cuarto día) $\binom{6}{6}$ (quinto día).
Por lo que la complejidad total será la suma de las complejidades de cada uno de estos días que es de es de 747,860 combinaciones diferentes que el algoritmo ha tenido en cuenta.

#Diseño
¿Que técnica utilizo? ¿Por qué?

La elección de un algoritmo heurístico, en este caso mediante una técnica  voraz (greedy)  se basa en varias características clave del problema:

-Complejidad y Tamaño del Problema: El problema tiene un gran espacio de soluciones debido a la cantidad de combinaciones posibles de tomas. Resolver esto de manera exacta por ejemplo, con programación lineal entera es muy costoso en términos de tiempo y recursos computacionales, como aparece en el primer algoritmo que he intentado implementar.

-Eficiencia en Tiempo de Ejecución: Los algoritmos greedy y los algoritmos heurísticos en general, son conocidos por su eficiencia en tiempo de ejecución. Priorizan la obtención de una solución buena y rápida sobre la búsqueda de la solución óptima, lo que es beneficioso en problemas con muchas variables o cuando se requieren respuestas en tiempos limitados.

-Simplicidad y Facilidad de Implementación: Los enfoques heurísticos suelen ser más sencillos de implementar y entender. El enfoque en este caso es claro: en cada paso, toma la mejor decisión posible en ese momento, sin preocuparse por el impacto global de esta decisión en el futuro del problema.

-Soluciones Localmente Óptimas: El algoritmo greedy busca soluciones que son localmente óptimas (es decir, la mejor solución posible en cada paso individual), lo que a menudo lleva a soluciones globalmente buenas, aunque no garantiza la optimalidad global.

  

-Adaptabilidad: Los algoritmos heurísticos pueden adaptarse fácilmente a cambios en las restricciones o en los criterios de evaluación, lo que puede ser útil si las condiciones del problema de planificación cambian